In [13]:
import os
import re
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
import transformers
from transformers import PreTrainedTokenizerFast
from transformers import LlamaConfig, LlamaModel, LlamaForCausalLM, TrainingArguments, DataCollatorForLanguageModeling, Trainer, TrainerCallback, AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, load_dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model


In [3]:
torch.cuda.is_available()

True

# Pretrain

### 1. Подготовка данных

In [4]:
novels_path ='../RussianNovels/corpus/'

In [5]:
texts = []

for file in os.listdir(novels_path):
    with open(novels_path + file, 'r', encoding='utf-8') as f:
        texts.append(f.read())

len(texts)

1

In [6]:
# Удаление дубликатов
texts = list(set(texts))
len(texts)

1

In [7]:
# Удаление предложений с буквами не из кириллицы
for i, text in enumerate(texts):
    sents_with_latin = re.findall(r'[^.!?]*[a-zA-Z]+[^.!?]*[.!?]', text)
    for sent in sents_with_latin:
        texts[i] = texts[i].replace(sent, '')

In [8]:
# Удаление повторяющейся пунктуации
for i, text in enumerate(texts):
    texts[i] = re.sub(r'([.,!?])\1+', r'\1', text)

In [9]:
# Деление текстов на чанки
texts_split = []

chunk_len = 1000 # Длина чанка (в символах)

for text in texts:
    l = int(np.ceil(len(text)/chunk_len))
    texts_split.append(text[0:chunk_len])
    for i in range(1,l):
        texts_split.append(text[chunk_len*i:chunk_len*i+chunk_len])

len(texts_split)

494

### 2. Создание токенизатора

In [10]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [11]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()

In [12]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=3000, special_tokens=special_tokens)

In [13]:
tokenizer.train_from_iterator(texts_split, trainer=trainer)

In [14]:
encoding = tokenizer.encode("Ну как, работает?")
print(encoding.tokens)

['ну', 'как', ',', 'рабо', '##та', '##ет', '?']


In [15]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [16]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [17]:
encoding = tokenizer.encode("Ну как, работает?")
print(encoding.tokens)

['[CLS]', 'ну', 'как', ',', 'рабо', '##та', '##ет', '?', '[SEP]']


In [18]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [19]:
tokenizer.decode(encoding.ids)

'ну как, работает?'

In [20]:
tokenizer.save("tokenizer.json")

In [21]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # В качестве альтернативы можно загрузить из файла токенизатора.
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

### 3. Создание Dataset-а

In [22]:
def tokenize_function(examples):
    return wrapped_tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset = Dataset.from_dict({"text": texts_split})
train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.remove_columns(["text"])

Map: 100%|██████████| 494/494 [00:00<00:00, 3173.30 examples/s]


In [23]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=wrapped_tokenizer,
    mlm=False,
)

### 4. Инициализация модели

In [24]:
config = LlamaConfig(
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    vocab_size=32000,
    max_position_embeddings=2048,
)

model = LlamaForCausalLM(config)

In [25]:
test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
]

In [26]:
# Коллбэк для валидации качества генерации
class ValidationCallback(TrainerCallback):
    def __init__(self, prompts, tokenizer, every_n_steps=500):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.every_n_steps = every_n_steps
        
    def on_step_end(self, args, state, control, model, **kwargs):
        if state.global_step % self.every_n_steps == 0 and state.global_step > 0:
            print(f"\n--- Валидация на шаге {state.global_step} ---")
            model.eval()
            with torch.no_grad():
                for i, prompt in enumerate(self.prompts):
                    inputs = self.tokenizer(prompt, return_tensors="pt")
                    
                    device = next(model.parameters()).device
                    inputs = {k: v.to(device) for k, v in inputs.items()}
                    
                    outputs = model.generate(
                        inputs.input_ids,
                        max_length=len(inputs.input_ids[0]) + 50,
                        num_return_sequences=1,
                        temperature=0.7,
                        do_sample=True,
                        pad_token_id=self.tokenizer.eos_token_id,
                        repetition_penalty=1.1
                    )
                    generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                    print(f"{i+1}. Промпт: {prompt}")
                    print(f"   Сгенерировано: {generated_text}")
            print("---")
            model.train()

In [ ]:
training_args = TrainingArguments(
    output_dir="./my_llm",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    save_steps=1000,
    save_total_limit=2,
    prediction_loss_only=True,
    remove_unused_columns=False,
    weight_decay=0.01,  # Регуляризация
    logging_steps=100,
)


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    callbacks=[ValidationCallback(test_prompts, wrapped_tokenizer, every_n_steps=500)]
)


In [29]:
trainer.train()

Step,Training Loss
100,7.526900
200,6.567100
300,6.352600


TrainOutput(global_step=372, training_loss=6.707455296670237, metrics={'train_runtime': 127.3697, 'train_samples_per_second': 11.635, 'train_steps_per_second': 2.921, 'total_flos': 448437532274688.0, 'train_loss': 6.707455296670237, 'epoch': 3.0})

In [30]:
model.eval()
results = []

for i, prompt in enumerate(test_prompts):
    inputs = wrapped_tokenizer(prompt, return_tensors="pt")
    
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'],
            max_length=len(inputs['input_ids'][0]) + 100,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=wrapped_tokenizer.eos_token_id,
            top_p=0.9,
            repetition_penalty=1.1
        )
    
    generated_text = wrapped_tokenizer.decode(outputs[0], skip_special_tokens=True)
    results.append({
        "№": i+1,
        "Промпт": prompt,
        "Сгенерированный текст": generated_text
    })
    
    print(f"/n{i+1}. ПРОМПТ: {prompt}")
    print(f"РЕЗУЛЬТАТ: {generated_text}")
    print("-" * 80, "/n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/n1. ПРОМПТ: Все мысли, которые имеют огромные последствия
РЕЗУЛЬТАТ: все мысли, которые имеют огромные последствиям. " на то, и обь и и но не турбин, что он в городе, а как я не то. - то? - и в так, как - да, по " и на прави. - это и не вь не выка. - и это, не не что - то, что? - вык, - ну, - то. - не то, - не, - то, а отя, - то. - нет. - я я
-------------------------------------------------------------------------------- /n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/n2. ПРОМПТ: Сила войска зависит от его духа
РЕЗУЛЬТАТ: сила воиска зависит от его духа, какемо на "н. - то? - а в не он и вот как и вот он. - сказал, как за город, - что у турбин, а не уь, а выко. - да, - то? - ну, - как - я это? - я это, я вы, - вы! - ну, - да. - я не не да, - я, - то, - с, - вь. - а у. - вы
-------------------------------------------------------------------------------- /n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

/n7. ПРОМПТ: Нет, жизнь не кончена
РЕЗУЛЬТАТ: нет, жизнь не кончена. - у, - вы не он? - я, и в него, - а? - то, - выите, - то - кто - по - сказал, - то, - то - что это. - я, - он! - я я же -, - то? - не на, - да, - то, а - а, - по - и не - си - я, - то, - и вот не. - нет, - ну, - и
-------------------------------------------------------------------------------- /n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/n8. ПРОМПТ: Всякая мысль, даже самая простая
РЕЗУЛЬТАТ: всякая мысль, даже самая простая в у и что - то - сказале на не а? - выно и то. - да, - не он? - с не вы, - в ка? - то. - яо, - уу, - ся, - ну, как - то, - то, - а - а же не ул, - нибудь? - и вот я? - то, - а - то, - вы? - наь? - то? - под. - в
-------------------------------------------------------------------------------- /n


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


/n9. ПРОМПТ: Война не любезность, а самое гадкое дело
РЕЗУЛЬТАТ: воина не любезность, а самое гадкое делои, а и за это не не, и а выа, что вьл. так, но на по его, и не раз не он. - то и на было, - у, - то и как, - то, и не не они, как я и вот - то, как же же не не вызо, - то, - то, что не, а не, - то? - то, - то, - сказал, - как не что " о
-------------------------------------------------------------------------------- /n
/n10. ПРОМПТ: Чтобы жить честно
РЕЗУЛЬТАТ: чтобы жить честно. и не при в, что но не это, и а так, как - ну, - не не не и не не. я, - то, - не не не него и вы, как не, - то. - то? - ск, - не поды, - то? - я, - и рази, - и, - нибудь, - вы. - что, - с. - и я турбин, - и не, - не то, что - то
-------------------------------------------------------------------------------- /n


# Post-train SFT

### 1. Подготовка датасета

In [12]:

def format_dialog(example):
    system_msg = example.get('input', '')
    user_msg = example.get('user', '')
    assistant_msg = example.get('assistant', '')
    
    # Формат Qwen chat template
    formatted_text = f"<|im_start|>system\n{system_msg}<|im_end|>\n<|im_start|>user\n{user_msg}<|im_end|>\n<|im_start|>assistant\n{assistant_msg}<|im_end|>"
    
    return {"text": formatted_text}

def prepare_alpaca_dataset(dataset):
    
    formatted_dataset = dataset.map(format_dialog)
    return formatted_dataset

dataset = load_dataset("d0rj/alpaca-cleaned-ru", split="train")
alpaca_dataset = prepare_alpaca_dataset(dataset)


Map: 100%|██████████| 51760/51760 [00:01<00:00, 30985.03 examples/s]


### 2. Дообучение модели

In [6]:
model_name = "Qwen/Qwen2.5-0.5B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [7]:
model.to('cuda')

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

training_args = TrainingArguments(
    output_dir="./qwen2.5-0.5b-alpaca",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    learning_rate=2e-4,
    bf16=True,
    logging_steps=10,
    save_steps=500,
    eval_steps=500,
    save_total_limit=2,
    load_best_model_at_end=False,
    report_to=None,
    ddp_find_unused_parameters=False,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=alpaca_dataset,
    peft_config=lora_config,
)

trainer.train()

trainer.save_model()

Truncating train dataset: 100%|██████████| 51760/51760 [00:00<00:00, 235298.71 examples/s]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,6.274100
20,4.686500
30,3.590500
40,3.033300


KeyboardInterrupt: 

### 3. Генерация ответов

In [20]:
questions_rus = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
  ] 

In [ ]:
for question in questions_rus:
    inputs = tokenizer(question, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    print('\nВОПРОС:', question)
    print("ОТВЕТ:", response, '\n')